# Imports

In [1]:
from agent_arena_no_gathering import *
from environments.evil_wgw_env import EvilWindyGridWorld

/root/miniconda3/envs/opensim-rl/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Agent training

In [2]:
num_actions = 4

In [ ]:
for i in range(5):
    for stoc in [0.0, 0.02, 0.05, 0.1, 0.2]:
        
        tf.reset_default_graph()
    
        dqn_trainer = DQNTrainer(
            num_actions, state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64, 64],
            gradient_clip=10.0,
            scope="dqn_trainer")

        quant_trainer = QuantileTrainer(
            num_actions, state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
            num_atoms=64, kappa=1.0,
            scope="qr_trainer")
        
        cat_trainer = CategoricalTrainer(
            num_actions, state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
            num_atoms=64, v=(-5, 5),
            scope="cat_trainer")

        trainers = [quant_trainer, dqn_trainer, cat_trainer]
        
        env = EvilWindyGridWorld(
            grid_size=(7, 10), visual=True, stochasticity=stoc)

        agent_arena = AgentArena(
            env, num_actions, trainers,
            save_path="evil_wgw_models/test/exp"+str(stoc)[2:]+"_"+str(i+1))

        agent_arena.set_parameters(
            replay_memory_size=100000,
            discount_factor=0.99,
            max_episode_length=100)
    
        agent_arena.start(gpu_id=4, max_num_frames=2000000)